## Part 1: Multinomial Naive Bayes- (multi-class text classification) on 20 news (DS1)



In [62]:
import numpy as np
import pandas as pd
import itertools
import matplotlib.pyplot as plt
from IPython.core.debugger import set_trace #for debugging
import string
np.random.seed(1234)
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize

#=======clean data=======
#remove difference between word with Capital first letter  and apple & appleS?
def remove_bad_characters(data):
  tokens = word_tokenize(data)
  # convert to lower case
  tokens = [w.lower() for w in tokens]
  # remove punctuation from each word
  import string
  table = str.maketrans('', '', string.punctuation)
  stripped = [w.translate(table) for w in tokens]
  # remove remaining tokens that are not alphabetic
  words = [word for word in stripped if word.isalpha()]
  # filter out stop words
  from nltk.corpus import stopwords
  stop_words = set(stopwords.words('english'))
  words = [w for w in words if not w in stop_words]
  sent = ' '.join(words)
  return sent

def num_doc_class(target):
  import collections
  counter=collections.Counter(target)
  print("Number of document per categories: {}".format(counter.values()))
  print("Categorie ID associated with previous numbers: {}".format(counter.keys()))
  counter_n=counter.values()

  for i in counter:
    counter[i]/=len(train.target)
  print("Probability of each class:")
  print("\n".join("{}: {}".format(k, v) for k, v in counter.items()))
    
  return counter,counter_n



[nltk_data] Downloading package punkt to /Users/muchderek/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/muchderek/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#20 news group dataset

In [63]:

##For 20news dataset
#======import dataset 20newsgroups=======
from sklearn.datasets import fetch_20newsgroups
twenty_newgroup=fetch_20newsgroups()

#=======load data ======= 
#categories=twenty_newgroup.target_names --> for all classes
categories=['alt.atheism', 'soc.religion.christian','comp.graphics'] #to test whithout long execution times
train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'), categories=categories) # F-score will be lower because it is more realistic
test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'), categories=categories)


#=======clean data ======= 
for i in range(len(train.data)): #for every doc 
    train.data[i]=remove_bad_characters(train.data[i])
for i in range(len(test.data)):
    test.data[i]=remove_bad_characters(test.data[i])
  
#=======count data frequency: word apearance in each doc======= #X_tfidf we can try with this later to see diff in accuracy
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(stop_words='english') #counts


X_train_count = count_vect.fit_transform(train.data) ## count of words in each email/doc for all classes
Unique_words_train= count_vect.get_feature_names()#reduces the text into a set of features
X_test_count = count_vect.transform(test.data)



#=======count data frequency:distriution of doc per class======= 
counter,counter_n=num_doc_class(train.target) 

y = train.target

#========Probability of each word per class=======
#Matrix_df=pd.DataFrame.sparse.from_spmatrix(X_train_count)
#testMatrix_df = pd.DataFrame.sparse.from_spmatrix(X_test_count)
#print(Matrix_df.iloc[:584,:]) #frequency of each word for each doc of class [1] 

#x_train = Matrix_df.to_numpy() ################# <===== THIS IS X_TRAIN BUT IN NUMPY PLS USE <=============================== ###########
#x_test = testMatrix_df.to_numpy()[:,np.isin(Unique_words_test, Unique_words_train)] #only keep columns of words contained in train set
#print(numMat[:584,:])

#=======data vizualization=======
print(twenty_newgroup.keys())
print("We have {} unique classes".format(len(categories)))
print("We have {} training samples".format(len(train.data))) # number of text sentences for training
print("We have {} test samples".format(len(test.data))) # number of text sentences for testing
print("Here are the 20 different classes {}".format(categories))
print("and their target number are {}".format(train.target))# the classes
#print("All words: {}".format(Unique_words_train)) #all the different words: numbers, signs and than alphabetic order 
print("The distribution of doc in each class: {}".format(counter))


Number of document per categories: dict_values([480, 599, 584])
Categorie ID associated with previous numbers: dict_keys([0, 2, 1])
Probability of each class:
0: 0.28863499699338546
2: 0.36019242333132895
1: 0.35117257967528565
dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])
We have 3 unique classes
We have 1663 training samples
We have 1106 test samples
Here are the 20 different classes ['alt.atheism', 'soc.religion.christian', 'comp.graphics']
and their target number are [0 2 2 ... 1 0 2]
The distribution of doc in each class: Counter({2: 0.36019242333132895, 1: 0.35117257967528565, 0: 0.28863499699338546})


 Implement multinomialnaive Bayes model as a Python class


a prior for the probability of each class by counting relative frequency of class labels in the dataset.
for multi-class classification, wich $C$ classes, this is a categorical distribution $\mathrm{Categorical}(y=c_0; \pi) = \prod_c \pi_c^{\mathbb{I}(y=c)} = \pi_{c_0}$. MLE is given by $\pi_c = \frac{N(y=c)}{N}$.
a probability for the value of each feature $x_d$, conditioned on the class label $y$. The type of this probability distribution depends on the feature type. 
Once we learn both the class prior $p(y)$ and class conditionals $p(x_d|y)$ from the data, we can use the Bayes rule to calculate posterior class probability $p(y| x) \propto p(y) p(x| y)$.

In [68]:
def logsumexp(Z):                                                # dimension C x N
      Zmax = np.max(Z)                              # max over C
      log_sum_exp = Zmax + np.log(np.sum(np.exp(Z - Zmax)))
      return log_sum_exp
    
class MultNaiveBayes:
    def __init__(self):
        return
    
    def fit(self, x, y, alpha, beta): # x is the document matrix, y is train.target
      N, D = x.shape # N = number of documents in total, D = words d (columns)
      self.C = np.max(y) + 1 # C is number of categories
      # one parameter for each feature conditioned on each class
      self.pi = np.zeros(self.C) # store prior values for each category
      Nc = np.zeros(self.C) # store number of instances in class c
      self.coeffs = np.zeros((self.C,D))
      
      # get the priors
      #import collections
      #DocsPerCat=collections.Counter(y)
      #print("Number of document per categories: {}".format(counter.values()))
      #print("Categorie ID associated with previous numbers: {}".format(counter.keys()))
      #for i in DocsPerCat:
       # self.pi[i]=(DocsPerCat[i]+alpha)/(len(y)+alpha+beta)
      
      # get likelihood coefficients
      for c in range(self.C): 
        self.pi[c]=(x[y==c].shape[0]+alpha)/(N+alpha+beta)
        wtot = np.sum(np.sum(x[y == c,:], axis = 0)) #get total label count of this category
        SumD = np.sum(x[y == c,:], axis = 0) #return a 1d array of sum of the elements of each column 
        self.coeffs[c,:] = (SumD+1)/(wtot+D) #get likelihood

      return self

    def predict(self, xt): #xt contains word d columns that exist in train set
        Nt, D = xt.shape #Nt is number of rows, ie number of test cases to predict, #D is number of words d (columns)
        y_prob = np.zeros(Nt)
        posterior = np.zeros((Nt, self.C)) #store posterior values for each category
        log_likelihood = np.zeros(self.C)

        log_prior = np.log(self.pi)
        for n in range(Nt):
            for c in range(self.C):
                #xtemp = xt[n,:]
                xtemp = xt[n].T
                x_=np.array(np.argwhere(xtemp > 0))
                x__=np.delete(x_,1,1)
                #log_likelihood[c] = np.sum(np.log(self.coeffs[c,np.isin(wordsTrain, wordsTest[xtemp[xtemp>0]])]))
                log_likelihood[c] = np.sum(np.log(self.coeffs[c,x__]))
                #log_likelihood = np.sum(log_likelihood, axis = 1) #perform sum across columns
            log_posterior = log_prior + log_likelihood
            posterior[n,:] = np.exp(log_posterior - logsumexp(log_posterior)) # normalization...; dim N x C
        return np.argmax(posterior,1) #or return y_pred directly by getting the argmax..

Experiment:
Apply model to classify documents

In [78]:
#y_train/test is train/test.target
import timeit
#X_train = X_train_count[0:int(X_train_count.shape[0] * train_size[each])]
y_train_less = train.target[0:int(train.target.shape[0]*1)] # try portions of data
#y_test = test.target[0:int(test.target.shape[0]*0.2)]
X_train_count_less = X_train_count[0:int(X_train_count.shape[0]*1)]
model = MultNaiveBayes()
t1 = timeit.default_timer()
model.fit(X_train_count_less, y_train_less, 1, 1)
train_time = timeit.default_timer() - t1
y_pred = model.predict(X_test_count)
accuracy = np.sum(y_pred == test.target)/y_pred.shape[0]
print(f'test accuracy: {accuracy}')

test accuracy: 0.8426763110307414


In [77]:
print(train.target.shape[0])
#1663 rows
#ok = np.delete(X_train_count, np.s_[1:100], axis = 0)
print(X_train_count.shape)
print(train.target.shape)
print(y_train.shape)
train.target.shape
print(X_train_count[0:int(X_train_count.shape[0]*1)].shape)

1663
(1663, 21083)
(1663,)
(332,)
(1663, 21083)


In [34]:
print(train_time)

0.2294095510005718


## Logistic Regression on DS1

In [46]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import timeit
from sklearn.datasets import fetch_20newsgroups_vectorized

def Logis_Regression(iters,norm,X_train,y_train,X_test,y_test,c):
  
    for this_max_iter in iters:

        lr=LogisticRegression(C=c,solver='saga',multi_class='multinomial',penalty=norm,max_iter=this_max_iter,random_state=0)
        t1 = timeit.default_timer()
        lr.fit(X_train,y_train)
        train_time = timeit.default_timer() - t1
        y_pred=lr.predict(X_test)

        #accuracy = np.sum(y_pred == y_test) / y_test.shape[0]
        accuracy=accuracy_score(y_test, y_pred)

    return accuracy,train_time

#=======load data ======= 
X_data, y_data = fetch_20newsgroups_vectorized(remove=('headers', 'footers', 'quotes'), return_X_y=True)
X_train, X_test, y_train, y_test=train_test_split(X_data,y_data,test_size=0.3)

#Import data set
#Hyperparameters
iters=[5.78]
norm='l1'
c=9.8
#calling function
Accuracy_regression_DS1,train_time=Logis_Regression(iters,norm,X_train,y_train,X_test,y_test,c)
print('Test accuracy for model {}: {}'.format('multinomial', Accuracy_regression_DS1*100))
print(train_time)

# from sklearn.model_selection import train_test_split, KFold, RandomizedSearchCV, cross_val_score
# from sklearn.preprocessing import LabelEncoder, OneHotEncoder
# kfold = KFold(n_splits=5, shuffle=True)
# lr=LogisticRegression(C=c,solver='saga',multi_class='multinomial',penalty=norm,max_iter=20,random_state=0)
# results = cross_val_score(lr, X_train, y_train, cv=kfold)
# print(results)


Test accuracy for model multinomial: 64.12371134020619
14.007876427000156
[0.64078283 0.63699495 0.62058081 0.66161616 0.63044852]


In [41]:
##TUNNING HYPERPARAMETERS 
C = np.linspace(0.5,10)
penalty = ['l1','l2']
iter = np.linspace(1,10)
hyperparameters = dict(C=C, penalty=penalty, max_iter=iter)
lr_tune = LogisticRegression()
rsearch = RandomizedSearchCV(lr_tune, hyperparameters, n_iter=10)
rsearch.fit(X_train,y_train)
print(rsearch.best_params_)

# #grid search
# from sklearn.model_selection import GridSearchCV
# lrGrid = LogisticRegression()
# C = np.linspace(0.5,10)
# penalty = ['l1','l2']
# iter = np.linspace(1,10)
# hyperparameters = dict(C=C, penalty=penalty, max_iter=iter)

# gridlr = GridSearchCV(lrGrid, hyperparameters, cv=5, verbose = 0)
# best_model = gridlr.fit(X_train,y_train)

/Users/muchderek/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/Users/muchderek/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/muchderek/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Users/muchderek/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


{'penalty': 'l1', 'max_iter': 5.775510204081633, 'C': 9.806122448979592}
